In [31]:
# ================== 1. Import ==================
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
import time, os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(">>> Đang sử dụng:", device)
if device.type == "cuda":
    print(">>> GPU:", torch.cuda.get_device_name(0))

# ================== 2. Data ==================
train_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# 👉 dataset bây giờ chỉ có 2 class: "pallet" và "nothing"
train_dir = "dataset_step1/images/train"
val_dir   = "dataset_step1/images/val"

train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset   = datasets.ImageFolder(val_dir,   transform=test_transform)
class_names = train_dataset.classes
print("Classes:", class_names)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# ================== 3. Model ==================
modelA = models.efficientnet_b0(pretrained=True)
num_ftrs = modelA.classifier[1].in_features
modelA.classifier[1] = nn.Linear(num_ftrs, len(class_names))
modelA = modelA.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(modelA.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=10)

# ================== 4. Training ==================
best_acc, patience, trigger = 0.0, 5, 0
for epoch in range(30):
    start = time.time()
    # Train
    modelA.train()
    train_loss, train_correct, train_total = 0, 0, 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = modelA(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
        train_correct += (out.argmax(1) == y).sum().item()
        train_total += y.size(0)
    train_acc = train_correct / train_total

    # Val
    modelA.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = modelA(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_correct += (out.argmax(1) == y).sum().item()
            val_total += y.size(0)
    val_acc = val_correct / val_total
    scheduler.step()

    print(f"Epoch {epoch+1:02d} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Time: {time.time()-start:.1f}s")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(modelA.state_dict(), "pallet_detector.pth")
        print("✅ Saved new best model")
        trigger = 0
    else:
        trigger += 1
        if trigger >= patience:
            print("⏹ Early stopping")
            break

print("Best Val Acc (Detector):", best_acc)


>>> Đang sử dụng: cuda:0
>>> GPU: NVIDIA GeForce RTX 3060 Laptop GPU
Classes: ['nothing', 'pallet']
Epoch 01 | Train Acc: 0.5000 | Val Acc: 0.3636 | Time: 9.6s
✅ Saved new best model
Epoch 02 | Train Acc: 0.8485 | Val Acc: 0.4545 | Time: 10.1s
✅ Saved new best model
Epoch 03 | Train Acc: 0.9697 | Val Acc: 0.5455 | Time: 9.8s
✅ Saved new best model
Epoch 04 | Train Acc: 0.9545 | Val Acc: 0.5455 | Time: 9.4s
Epoch 05 | Train Acc: 0.9697 | Val Acc: 0.4545 | Time: 9.7s
Epoch 06 | Train Acc: 0.9848 | Val Acc: 0.5455 | Time: 10.9s
Epoch 07 | Train Acc: 1.0000 | Val Acc: 0.6364 | Time: 9.8s
✅ Saved new best model
Epoch 08 | Train Acc: 0.9697 | Val Acc: 0.6364 | Time: 9.6s
Epoch 09 | Train Acc: 1.0000 | Val Acc: 0.6364 | Time: 9.7s
Epoch 10 | Train Acc: 0.9848 | Val Acc: 0.6364 | Time: 9.5s
Epoch 11 | Train Acc: 1.0000 | Val Acc: 0.8182 | Time: 9.8s
✅ Saved new best model
Epoch 12 | Train Acc: 0.9848 | Val Acc: 0.7273 | Time: 9.7s
Epoch 13 | Train Acc: 0.9848 | Val Acc: 0.7273 | Time: 9.9s
Epo

In [ ]:
# ================== 1. Import ==================
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
import time, os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(">>> Đang sử dụng:", device)
if device.type == "cuda":
    print(">>> GPU:", torch.cuda.get_device_name(0))

# ================== 2. Data ==================
train_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# 👉 dataset bây giờ chỉ có 2 class: "empty_pallet", "loaded_pallet"
train_dir = "dataset_step2/images/train"
val_dir   = "dataset_step2/images/val"

train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset   = datasets.ImageFolder(val_dir,   transform=test_transform)
class_names = train_dataset.classes
print("Classes:", class_names)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# ================== 3. Model ==================
modelB = models.efficientnet_b0(pretrained=True)
num_ftrs = modelB.classifier[1].in_features
modelB.classifier[1] = nn.Linear(num_ftrs, len(class_names))
modelB = modelB.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(modelB.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=10)

# ================== 4. Training ==================
best_acc, patience, trigger = 0.0, 5, 0
for epoch in range(30):
    start = time.time()
    # Train
    modelB.train()
    train_loss, train_correct, train_total = 0, 0, 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = modelB(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
        train_correct += (out.argmax(1) == y).sum().item()
        train_total += y.size(0)
    train_acc = train_correct / train_total

    # Val
    modelB.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = modelB(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_correct += (out.argmax(1) == y).sum().item()
            val_total += y.size(0)
    val_acc = val_correct / val_total
    scheduler.step()

    print(f"Epoch {epoch+1:02d} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Time: {time.time()-start:.1f}s")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(modelB.state_dict(), "pallet_classifier.pth")
        print("✅ Saved new best model")
        trigger = 0
    else:
        trigger += 1
        if trigger >= patience:
            print("⏹ Early stopping")
            break

print("Best Val Acc (Classifier):", best_acc)


>>> Đang sử dụng: cuda:0
>>> GPU: NVIDIA GeForce RTX 3060 Laptop GPU
Classes: ['empty_pallet', 'loaded_pallet']
Epoch 01 | Train Acc: 0.5294 | Val Acc: 0.7500 | Time: 10.5s
✅ Saved new best model
Epoch 02 | Train Acc: 0.6275 | Val Acc: 0.7750 | Time: 10.6s
✅ Saved new best model
Epoch 03 | Train Acc: 0.8235 | Val Acc: 0.7750 | Time: 10.8s
Epoch 04 | Train Acc: 0.8431 | Val Acc: 0.7750 | Time: 10.4s
Epoch 05 | Train Acc: 0.9020 | Val Acc: 0.7750 | Time: 12.2s
Epoch 06 | Train Acc: 0.9804 | Val Acc: 0.7250 | Time: 13.9s
Epoch 07 | Train Acc: 0.9608 | Val Acc: 0.7500 | Time: 10.8s
⏹ Early stopping
Best Val Acc (Classifier): 0.775


In [33]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# ---- Step 1: Load models ----
def load_model_efficientnet(num_classes, weight_path):
    model = models.efficientnet_b0(pretrained=False)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, num_classes)
    state_dict = torch.load(weight_path, map_location="cpu")
    model.load_state_dict(state_dict)
    model.eval()
    return model

# model1: Có pallet hay không
model1 = load_model_efficientnet(2, "pallet_detector.pth")

# model2: Empty pallet vs Loaded pallet
model2 = load_model_efficientnet(2, "pallet_classifier.pth")

# ---- Step 2: Preprocessing ----
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ---- Step 3: Inference Pipeline ----
def infer_pipeline(image_path):
    img = Image.open(image_path).convert("RGB")
    x = transform(img).unsqueeze(0)  # (1, C, H, W)

    with torch.no_grad():
        # Step 1: Pallet vs Nothing
        out1 = model1(x)
        prob1 = torch.softmax(out1, dim=1)
        pred1 = torch.argmax(prob1, dim=1).item()

    if pred1 == 0:  
        return {"label": "No Pallet", "confidence": prob1[0][0].item()}
    else:
        with torch.no_grad():
            # Step 2: Empty vs Loaded
            out2 = model2(x)
            prob2 = torch.softmax(out2, dim=1)
            pred2 = torch.argmax(prob2, dim=1).item()

        if pred2 == 0:
            return {"label": "Empty Pallet", "confidence": prob2[0][0].item()}
        else:
            return {"label": "Loaded Pallet", "confidence": prob2[0][1].item()}

# ---- Test thử ----
for i in range (1, 9):
    result = infer_pipeline(f"D:\LAB\{i}.jpg")
    print("Result:", result)
# result = infer_pipeline(r"D:\LAB\nothing.jpg")
# print("Result:", result)

<string>:57: SyntaxWarning: invalid escape sequence '\{'
<>:57: SyntaxWarning: invalid escape sequence '\{'
<>:57: SyntaxWarning: invalid escape sequence '\L'
<string>:57: SyntaxWarning: invalid escape sequence '\{'
<>:57: SyntaxWarning: invalid escape sequence '\{'
<>:57: SyntaxWarning: invalid escape sequence '\L'
C:\Users\ducho\AppData\Local\Temp\ipykernel_15884\2707743042.py:57: SyntaxWarning: invalid escape sequence '\{'
  result = infer_pipeline(f"D:\LAB\{i}.jpg")
C:\Users\ducho\AppData\Local\Temp\ipykernel_15884\2707743042.py:57: SyntaxWarning: invalid escape sequence '\L'
  result = infer_pipeline(f"D:\LAB\{i}.jpg")


Result: {'label': 'Loaded Pallet', 'confidence': 0.5987045168876648}
Result: {'label': 'Loaded Pallet', 'confidence': 0.6071726679801941}
Result: {'label': 'Loaded Pallet', 'confidence': 0.651948094367981}
Result: {'label': 'Loaded Pallet', 'confidence': 0.7191992998123169}
Result: {'label': 'Loaded Pallet', 'confidence': 0.6772565245628357}
Result: {'label': 'Loaded Pallet', 'confidence': 0.6772565245628357}
Result: {'label': 'Loaded Pallet', 'confidence': 0.7914567589759827}
Result: {'label': 'No Pallet', 'confidence': 0.5124083161354065}
